In [ ]:
import requests
import pandas as pd
import time
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, Lipinski
import warnings
warnings.filterwarnings('ignore')

class ChEMBLHOBExtractor:
    def __init__(self):
        self.base_url = "https://www.ebi.ac.uk/chembl/api/data"
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Python ChEMBL HOB Extractor',
            'Accept': 'application/json'
        })
    
    def get_hob_activities(self, limit=1000, offset=0):
        """fetch activities related to human oral bioavailability"""
        url = f"{self.base_url}/activity.json"
        
        # specific hob-related standard types
        hob_types = [
            'F',                           # Bioavailability fraction
            '%F',                          # Bioavailability percentage  
            'Bioavailability',             # Explicit bioavailability
            'Oral bioavailability',        # Oral bioavailability
            'F (%)',                       # Bioavailability in percentage
            'Bioavailable fraction',       # Bioavailable fraction
            'BA',                          # Bioavailability abbreviation
            'Oral BA',                     # Oral bioavailability abbreviation
            'Absolute bioavailability',    # Absolute bioavailability
            'Oral F',                      # Oral bioavailability fraction
            'F%',                          # Alternative percentage format
            'Frel',                        # Relative bioavailability
            'AUC ratio'                    # Sometimes used for bioavailability
        ]
        
        params = {
            'standard_type__in': ','.join(hob_types),
            'organism': 'Homo sapiens',
            'limit': limit,
            'offset': offset,
            'format': 'json'
        }
        
        try:
            response = self.session.get(url, params=params, timeout=30)
            response.raise_for_status()
            data = response.json()
            
            if 'activities' in data:
                print(f"Retrieved {len(data['activities'])} activities from API")
            
            return data
        except Exception as e:
            print(f"error fetching activities: {e}")
            return None
    
    def is_valid_hob_assay(self, activity):
        """validation for oral bioavailability assays"""
        
        # Get assay and activity details
        assay_desc = activity.get('assay_description', '').lower()
        standard_type = activity.get('standard_type', '').lower()
        document_journal = activity.get('document_journal', '').lower()
        assay_organism = activity.get('assay_organism', '').lower()
        
        # human studies
        if 'homo sapiens' not in assay_organism and 'human' not in assay_desc:
            return False, "Not human study"
        
        # exclude non pk assays
        exclude_terms = [
            'receptor', 'binding', 'inhibition', 'ic50', 'ki', 'kd', 'enzyme',
            'cytotoxicity', 'cell death', 'cell line', 'growth', 'proliferation', 'viability',
            'antimicrobial', 'antibacterial', 'antifungal', 'antiviral', 'mic', 'mic90',
            'channel', 'transporter', 'substrate', 'metabolism', 'metabolite',
            'clearance', 'half-life', 'protein binding', 'solubility', 'permeability',
            'caco-2', 'mdck', 'pampa', 'in vitro', 'cell-based', 'functional',
            'agonist', 'antagonist', 'selectivity', 'potency', 'efficacy',
            'dose-response', 'ec50', 'screening', 'hts', 'functional assay'
        ]
        
        for term in exclude_terms:
            if term in assay_desc:
                return False, f"Excluded: contains '{term}'"
        
        # ind for hob
        pk_indicators = [
            'bioavailability', 'oral', 'pharmacokinetic', 'absorption', 'pk study',
            'plasma', 'serum', 'blood level', 'systemic exposure', 'auc',
            'oral administration', 'oral dose', 'po administration', 'per os',
            'clinical trial', 'human volunteer', 'phase i', 'phase ii',
            'crossover', 'bioequivalence', 'pharmacokinetics'
        ]
        
        clear_hob_types = [
            'f', '%f', 'bioavailability', 'oral bioavailability', 'ba', 'oral ba',
            'absolute bioavailability', 'oral f', 'f%', 'f (%)'
        ]
        
        if standard_type in clear_hob_types:
            # check pk context
            if any(indicator in assay_desc for indicator in pk_indicators):
                return True, f"Clear HOB type + PK context"
            elif len(assay_desc) < 50:  # Short descriptions might miss context
                return True, "Clear HOB type (short description)"
            else:
                return False, "Clear HOB type but no PK context in description"
        
        # For other types, require strong PK indicators
        pk_matches = [ind for ind in pk_indicators if ind in assay_desc]
        if len(pk_matches) >= 2:  # Multiple PK indicators increase confidence
            return True, f"Multiple PK indicators: {', '.join(pk_matches[:2])}"
        elif any(strong_indicator in assay_desc for strong_indicator in ['oral bioavailability', 'absolute bioavailability', 'pharmacokinetic study']):
            return True, "Strong PK indicator found"
        
        return False, "Insufficient PK indicators"
    
    def validate_hob_value(self, value, units, standard_type):
        """Enhanced validation and normalization of bioavailability values"""
        
        try:
            hob_value = float(value)
        except (ValueError, TypeError):
            return None, "Invalid numeric value"
        
        # Handle different unit systems more carefully
        units_str = str(units).lower() if units else ''
        
        if '%' in units_str or 'percent' in units_str:
            # Already in percentage
            normalized_value = hob_value
        elif units_str in ['', 'nan', 'none', 'unspecified'] or units is None:
            # No units specified - infer based on value and type
            if 'f' in standard_type.lower() and hob_value <= 1.0:
                # Bioavailability fraction (0-1), convert to percentage
                normalized_value = hob_value * 100
            elif hob_value <= 1.0:
                # Likely a fraction
                normalized_value = hob_value * 100
            elif 1.0 < hob_value <= 100.0:
                # Likely already percentage
                normalized_value = hob_value
            else:
                # Values >100% need careful handling
                if hob_value > 200:
                    return None, f"Unrealistic value: {hob_value}"
                normalized_value = hob_value
        elif 'fraction' in units_str or 'ratio' in units_str:
            # Fraction format
            normalized_value = hob_value * 100 if hob_value <= 1.0 else hob_value
        else:
            # Unknown units - be conservative
            if hob_value <= 1.0:
                normalized_value = hob_value * 100
            else:
                normalized_value = hob_value
        
        # Final validation with more realistic bounds
        if normalized_value < 0:
            return None, "Negative bioavailability"
        elif normalized_value > 150:  # Allow for some measurement error and relative bioavailability
            return None, f"Unrealistic bioavailability: {normalized_value}%"
        elif normalized_value == 0:
            return None, "Zero bioavailability (likely data error)"
        
        return round(normalized_value, 2), "Valid"
    
    def get_molecule_details(self, chembl_id):
        """Get molecule details including SMILES and properties"""
        url = f"{self.base_url}/molecule/{chembl_id}.json"
        
        try:
            response = self.session.get(url, timeout=30)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Error fetching molecule {chembl_id}: {e}")
            return None
    
    def calculate_descriptors(self, smiles):
        """Calculate molecular descriptors using RDKit"""
        try:
            mol = Chem.MolFromSmiles(smiles)
            if mol is None:
                return None
            
            descriptors = {
                'hacc': Descriptors.NumHAcceptors(mol),
                'hdon': Descriptors.NumHDonors(mol),
                'mw': Descriptors.MolWt(mol),
                'logp': Crippen.MolLogP(mol),
                'mr': Crippen.MolMR(mol),
                'tpsa': Descriptors.TPSA(mol),
                'nrot': Descriptors.NumRotatableBonds(mol),
                'num_rings': Descriptors.RingCount(mol),
                'aromatic_rings': Descriptors.NumAromaticRings(mol),
                'hba': Descriptors.NOCount(mol),  # N and O atoms
                'rule_of_five_violations': 0  # Will calculate
            }
            
            # Calculate Rule of Five violations
            violations = 0
            if descriptors['mw'] > 500: violations += 1
            if descriptors['logp'] > 5: violations += 1
            if descriptors['hdon'] > 5: violations += 1
            if descriptors['hacc'] > 10: violations += 1
            descriptors['rule_of_five_violations'] = violations
            
            return descriptors
        except Exception as e:
            print(f"Error calculating descriptors for {smiles}: {e}")
            return None
    
    def extract_hob_data(self, max_compounds=2000):
        """Extract high-quality human oral bioavailability data"""
        print("="*70)
        print("CHEMBL HUMAN ORAL BIOAVAILABILITY (HOB) EXTRACTOR v2.0")
        print("="*70)
        print("Filtering for authentic pharmacokinetic studies only...")
        
        all_data = []
        offset = 0
        limit = 1000
        total_processed = 0
        total_excluded = 0
        exclusion_reasons = {}
        
        while total_processed < max_compounds:
            print(f"\nFetching batch {offset//limit + 1} (offset: {offset})...")
            
            # Get activities
            activities_data = self.get_hob_activities(limit=limit, offset=offset)
            
            if not activities_data or 'activities' not in activities_data:
                print("No more activities found.")
                break
            
            activities = activities_data['activities']
            if not activities:
                print("Empty batch - stopping.")
                break
            
            print(f"Processing {len(activities)} activities...")
            
            for activity in activities:
                try:
                    # Extract basic info
                    chembl_id = activity.get('molecule_chembl_id')
                    standard_value = activity.get('standard_value')
                    standard_type = activity.get('standard_type')
                    standard_units = activity.get('standard_units')
                    
                    if not chembl_id or standard_value is None:
                        continue
                    
                    # Enhanced validation for HOB assays
                    is_valid, reason = self.is_valid_hob_assay(activity)
                    if not is_valid:
                        total_excluded += 1
                        exclusion_reasons[reason] = exclusion_reasons.get(reason, 0) + 1
                        continue
                    
                    # Enhanced validation and normalization
                    hob_value, validation_msg = self.validate_hob_value(
                        standard_value, standard_units, standard_type
                    )
                    
                    if hob_value is None:
                        total_excluded += 1
                        exclusion_reasons[validation_msg] = exclusion_reasons.get(validation_msg, 0) + 1
                        continue
                    
                    # Check for duplicates
                    if any(data['chembl_id'] == chembl_id for data in all_data):
                        continue
                    
                    # Get molecule details
                    molecule_data = self.get_molecule_details(chembl_id)
                    if not molecule_data:
                        continue
                    
                    # Extract molecule info
                    pref_name = molecule_data.get('pref_name', 'Unknown')
                    max_phase = molecule_data.get('max_phase', 0)
                    
                    # Get SMILES
                    smiles = None
                    structures = molecule_data.get('molecule_structures')
                    if structures:
                        smiles = structures.get('canonical_smiles')
                    
                    if not smiles:
                        total_excluded += 1
                        exclusion_reasons['No SMILES'] = exclusion_reasons.get('No SMILES', 0) + 1
                        continue
                    
                    # Calculate descriptors
                    descriptors = self.calculate_descriptors(smiles)
                    if not descriptors:
                        total_excluded += 1
                        exclusion_reasons['Descriptor calculation failed'] = exclusion_reasons.get('Descriptor calculation failed', 0) + 1
                        continue
                    
                    # Compile final data
                    compound_data = {
                        'chembl_id': chembl_id,
                        'drug_name': pref_name,
                        'smiles': smiles,
                        'hob': hob_value,  # Human oral bioavailability (%)
                        'max_phase': max_phase,
                        'measurement_type': standard_type,
                        'original_value': float(standard_value),
                        'original_units': standard_units or 'unspecified',
                        'validation_reason': reason,
                        # Molecular descriptors
                        'mw': round(descriptors['mw'], 2),
                        'logp': round(descriptors['logp'], 2),
                        'hacc': descriptors['hacc'],
                        'hdon': descriptors['hdon'],
                        'tpsa': round(descriptors['tpsa'], 2),
                        'nrot': descriptors['nrot'],
                        'mr': round(descriptors['mr'], 2),
                        'num_rings': descriptors['num_rings'],
                        'aromatic_rings': descriptors['aromatic_rings'],
                        'ro5_violations': descriptors['rule_of_five_violations']
                    }
                    
                    all_data.append(compound_data)
                    total_processed += 1
                    
                    if total_processed % 10 == 0:
                        print(f"✓ {total_processed} valid HOB compounds extracted")
                    
                    # Rate limiting
                    time.sleep(0.1)
                    
                except Exception as e:
                    print(f"Error processing activity: {e}")
                    continue
            
            offset += limit
            
            # Stop if we got fewer results than requested
            if len(activities) < limit:
                print("Reached end of available data.")
                break
        
        # Print exclusion summary
        print(f"\n" + "="*70)
        print("EXTRACTION SUMMARY")
        print("="*70)
        print(f"✓ Valid HOB compounds extracted: {len(all_data)}")
        print(f"✗ Total excluded entries: {total_excluded}")
        
        if exclusion_reasons:
            print(f"\nTop exclusion reasons:")
            for reason, count in sorted(exclusion_reasons.items(), key=lambda x: x[1], reverse=True)[:10]:
                print(f"  • {reason}: {count}")
        
        return all_data
    
    def save_hob_dataset(self, data, filename="hob_dataset_clean.csv"):
        """Save HOB dataset with enhanced quality metrics"""
        
        if not data:
            print("No data to save!")
            return None
        
        df = pd.DataFrame(data)
        
        # Remove duplicates based on SMILES (keep the one with higher max_phase)
        df_dedup = df.sort_values('max_phase', ascending=False).drop_duplicates('smiles', keep='first')
        print(f"Removed {len(df) - len(df_dedup)} duplicates based on SMILES")
        
        # Sort by HOB value (descending)
        df_final = df_dedup.sort_values('hob', ascending=False).reset_index(drop=True)
        
        # Reorder columns for better usability
        column_order = [
            'chembl_id', 'smiles', 'hob',  # Core data
            'mw', 'logp', 'hacc', 'hdon', 'tpsa', 'nrot', 'mr',  # Key descriptors
            'num_rings', 'aromatic_rings', 'ro5_violations',  # Additional descriptors
            'drug_name', 'max_phase', 'measurement_type', 'original_value', 'original_units', 'validation_reason'  # Metadata
        ]
        
        df_final = df_final[column_order].copy()
        
        # Save dataset
        df_final.to_csv(filename, index=False)
        
        # Generate comprehensive statistics
        stats = {
            'total_compounds': len(df_final),
            'unique_smiles': df_final['smiles'].nunique(),
            'hob_range': f"{df_final['hob'].min():.1f} - {df_final['hob'].max():.1f}%",
            'hob_mean': f"{df_final['hob'].mean():.1f}%",
            'hob_median': f"{df_final['hob'].median():.1f}%",
            'hob_std': f"{df_final['hob'].std():.1f}%",
            'clinical_compounds': len(df_final[df_final['max_phase'] >= 1]),
            'approved_drugs': len(df_final[df_final['max_phase'] == 4])
        }
        
        print(f"\n" + "="*60)
        print("HOB DATASET STATISTICS")
        print("="*60)
        for key, value in stats.items():
            print(f"{key.replace('_', ' ').title()}: {value}")
        
        # HOB distribution analysis
        print(f"\nHOB Distribution:")
        print(f"  0-20%:    {len(df_final[(df_final['hob'] >= 0) & (df_final['hob'] <= 20)])} compounds ({len(df_final[(df_final['hob'] >= 0) & (df_final['hob'] <= 20)])/len(df_final)*100:.1f}%)")
        print(f"  20-50%:   {len(df_final[(df_final['hob'] > 20) & (df_final['hob'] <= 50)])} compounds ({len(df_final[(df_final['hob'] > 20) & (df_final['hob'] <= 50)])/len(df_final)*100:.1f}%)")
        print(f"  50-80%:   {len(df_final[(df_final['hob'] > 50) & (df_final['hob'] <= 80)])} compounds ({len(df_final[(df_final['hob'] > 50) & (df_final['hob'] <= 80)])/len(df_final)*100:.1f}%)")
        print(f"  80-100%:  {len(df_final[(df_final['hob'] > 80) & (df_final['hob'] <= 100)])} compounds ({len(df_final[(df_final['hob'] > 80) & (df_final['hob'] <= 100)])/len(df_final)*100:.1f}%)")
        print(f"  >100%:    {len(df_final[df_final['hob'] > 100])} compounds ({len(df_final[df_final['hob'] > 100])/len(df_final)*100:.1f}%)")
        
        # Drug development phase analysis
        print(f"\nDevelopment Phase Distribution:")
        for phase in sorted(df_final['max_phase'].unique()):
            count = len(df_final[df_final['max_phase'] == phase])
            phase_name = {0: 'Preclinical', 1: 'Phase I', 2: 'Phase II', 3: 'Phase III', 4: 'Approved'}.get(phase, f'Phase {phase}')
            print(f"  {phase_name}: {count} compounds ({count/len(df_final)*100:.1f}%)")
        
        print(f"\nDataset saved as: {filename}")
        print(f"🎯 Ready for HOB prediction modeling!")
        
        return df_final

def main():
    """Main execution function"""
    print("Starting Enhanced HOB Dataset Extraction...")
    
    # Initialize extractor
    extractor = ChEMBLHOBExtractor()
    
    # Extract HOB data with higher limit
    print("Targeting high-quality human oral bioavailability data...")
    hob_data = extractor.extract_hob_data(max_compounds=1500)
    
    # Save dataset
    if hob_data:
        df = extractor.save_hob_dataset(hob_data)
        
    else:
        print("❌ No valid HOB data extracted!")
        print("Consider:")
        print("  • Expanding search terms")
        print("  • Relaxing validation criteria")
        print("  • Checking ChEMBL API connectivity")

if __name__ == "__main__":
    main()

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("/home/rizanb/Documents/hob_pred/data/raw/hob_dataset_clean.csv")
df.head()

,chembl_id,smiles,hob,mw,logp,hacc,hdon,tpsa,nrot,mr,num_rings,aromatic_rings,ro5_violations,drug_name,max_phase,measurement_type,original_value,original_units,validation_reason
0,CHEMBL4874698,CC(C)(O)c1cc(F)c2c(c1)C(=O)N(Cc1ccc(Cl)cn1)[C@...,150.0,558.44,4.89,5,2,105.75,8,139.81,5,3,1,NaN,NaN,F,150.0,%,Clear HOB type + PK context
1,CHEMBL4848062,COc1nc(C2=NOC(C3CC3)C(c3ccc(Cl)cc3)N2)ccc1-n1c...,113.0,423.90,4.04,7,1,73.56,5,114.11,5,3,0,NaN,NaN,F,113.0,%,Clear HOB type + PK context
2,CHEMBL2325014,Nc1[nH]ncc1-c1cc(Cl)ccc1Oc1cc(F)c(S(=O)(=O)Nc2...,110.0,500.36,5.15,7,3,122.99,6,117.85,4,4,2,PF-05089771,2.0,F,110.0,%,Clear HOB type + PK context
3,CHEMBL2324776,O=S(=O)(Nc1nncs1)c1cc(Cl)c(Oc2ccc(C(F)(F)F)cc2...,101.0,531.90,5.40,8,1,106.96,6,114.67,4,4,2,NaN,NaN,F,101.0,%,Clear HOB type + PK context
4,CHEMBL1009,N[C@@H](Cc1ccc(O)c(O)c1)C(=O)O,100.0,197.19,0.05,4,4,103.78,3,49.09,1,1,0,LEVODOPA,4.0,F,100.0,%,Clear HOB type + PK context


In [7]:
df.describe()

,hob,mw,logp,hacc,hdon,tpsa,nrot,mr,num_rings,aromatic_rings,ro5_violations,max_phase,original_value
count,722.00000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,722.000000,533.000000,722.00000
mean,58.48633,365.917022,2.578657,5.027701,1.939058,82.572770,5.042936,98.599211,3.045706,1.851801,0.297784,3.697936,58.48633
std,29.47186,159.915040,2.030572,2.939861,1.604138,50.277824,3.527123,42.311503,1.595543,1.158184,0.683647,0.845563,29.47186
min,0.20000,46.070000,-8.160000,1.000000,0.000000,3.240000,0.000000,12.760000,0.000000,0.000000,0.000000,-1.000000,0.20000
25%,33.00000,262.352500,1.452500,3.000000,1.000000,46.990000,3.000000,73.010000,2.000000,1.000000,0.000000,4.000000,33.00000
50%,69.00000,330.750000,2.580000,4.000000,2.000000,72.720000,4.000000,90.120000,3.000000,2.000000,0.000000,4.000000,69.00000
75%,80.00000,431.232500,3.790000,6.000000,3.000000,106.192500,7.000000,114.212500,4.000000,2.000000,0.000000,4.000000,80.00000
max,150.00000,1736.180000,10.810000,28.000000,12.000000,340.120000,27.000000,450.180000,14.000000,6.000000,4.000000,4.000000,150.00000
